In [ ]:
import anndata as ad
import numpy as np
import scanpy as sc
import pandas as pd
import openpyxl
import csv
import matplotlib
import scipy.stats as sp
import xgboost as xgb
import pickle
from sklearn.metrics import confusion_matrix, adjusted_rand_score
import matplotlib.pyplot as plt
from matplotlib import gridspec
import scrublet as scr
import seaborn as sns

In [ ]:
#N1 SC
N1_SC = sc.read_10x_mtx('H:/data1/N1-SC/outs/filtered_feature_bc_matrix',var_names='gene_symbols', cache=False)
print(N1_SC)
N1_SC.write_h5ad('F:/single cell data/SC/N1-SC.h5ad')

In [ ]:
N1_SC=sc.read_h5ad('F:/single cell data/SC/N1-SC.h5ad')
sc.pp.filter_cells(N1_SC, min_genes=200)#cell must have 200 non-zero-count features to stay
sc.pp.filter_genes(N1_SC, min_cells= 8)#gene must be in 5 cells to stay
mito_genes = N1_SC.var_names.str.startswith('mt-')# for each cell compute fraction of counts in mito genes vs. all genes
N1_SC.obs['percent_mito'] = np.sum(N1_SC[:, mito_genes].X, axis =1)/np.sum(N1_SC.X, axis=1) # add each cell's fraction mito accounts as an obs annotation
N1_SC.obs['n_counts'] =N1_SC.X.sum(axis=1) # add the total counts per cell as observations-annotation
N1_SC = N1_SC[N1_SC.obs.percent_mito < 0.01,:]
N1_SC = N1_SC[N1_SC.obs['n_genes']< 4000,:]
N1_SC = N1_SC[N1_SC.obs['n_counts'] < 10000, :]

In [ ]:
#Visualize count matrix before filtering
sc.pl.violin(N1_SC, ['n_genes', 'n_counts','percent_mito'], jitter =0.4, multi_panel= True)
sc.pl.scatter(N1_SC, x= 'n_counts', y= 'percent_mito')
sc.pl.scatter(N1_SC, x= 'n_counts', y= 'n_genes')
counts_matrix = N1_SC.X
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate= 0.06)
doublet_scores, precdicted_doublets = scrub.scrub_doublets(min_cells= 8, min_gene_variability_pctl= 85, n_prin_comps=40)
N1_SC.obs['Doublet'] = scrub.predicted_doublets_
N1_SC.obs['Doublet Score'] = scrub.doublet_scores_obs_
N1_SC = N1_SC[N1_SC.obs['Doublet']== False,:]
N1_SC.write_h5ad('F:/single cell data/SC/N1-SC_filter.h5ad')
print(N1_SC)

In [ ]:
# N2-SC
N2_SC = sc.read_10x_mtx('H:/data1/N2-SC/outs/filtered_feature_bc_matrix',var_names='gene_symbols', cache=False)
print(N2_SC)
N2_SC.write_h5ad('F:/single cell data/SC/N2-SC.h5ad')

In [ ]:
sc.pp.filter_cells(N2_SC, min_genes=200)#cell must have 200 non-zero-count features to stay
sc.pp.filter_genes(N2_SC, min_cells= 8)#gene must be in 5 cells to stay
mito_genes = N2_SC.var_names.str.startswith('mt-')# for each cell compute fraction of counts in mito genes vs. all genes
N2_SC.obs['percent_mito'] = np.sum(N2_SC[:, mito_genes].X, axis =1)/np.sum(N2_SC.X, axis=1) # add each cell's fraction mito accounts as an obs annotation
N2_SC.obs['n_counts'] =N2_SC.X.sum(axis=1) # add the total counts per cell as observations-annotation
N2_SC = N2_SC[N2_SC.obs.percent_mito < 0.01,:]
N2_SC = N2_SC[N2_SC.obs['n_genes']< 4000,:]
N2_SC = N2_SC[N2_SC.obs['n_counts'] < 10000, :]
print(N2_SC)

In [ ]:
sc.pl.violin(N2_SC, ['n_genes', 'n_counts','percent_mito'], jitter =0.4, multi_panel= True)
sc.pl.scatter(N2_SC, x= 'n_counts', y= 'percent_mito')
sc.pl.scatter(N2_SC, x= 'n_counts', y= 'n_genes')
counts_matrix = N2_SC.X
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate= 0.06)
doublet_scores, precdicted_doublets = scrub.scrub_doublets(min_cells= 8, min_gene_variability_pctl= 85, n_prin_comps=40)
N2_SC.obs['Doublet'] = scrub.predicted_doublets_
N2_SC.obs['Doublet Score'] = scrub.doublet_scores_obs_
N2_SC = N2_SC[N2_SC.obs['Doublet']== False,:]
N2_SC.write_h5ad('F:/single cell data/SC/N2-SC_filter.h5ad')
print(N2_SC)

In [ ]:
#N1_SC&N2_SC
N1_SC=sc.read_h5ad('F:/single cell data/SC/N1-SC_filter.h5ad')
N2_SC=sc.read_h5ad('F:/single cell data/SC/N2-SC_filter.h5ad')
N_SC =sc.AnnData.concatenate(N1_SC, N2_SC,batch_categories=['N1_SC','N2_SC'],batch_key='sample')
print(N_SC)

In [ ]:
sc.pp.normalize_per_cell(N_SC,counts_per_cell_after= 1e4)
sc.pp.log1p(N_SC)
sc.pp.highly_variable_genes(N_SC, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.tl.pca(N_SC,svd_solver='arpack')
sc.external.pp.harmony_integrate(N_SC,key='sample')
sc.pp.neighbors(N_SC,n_neighbors=25, n_pcs=40,use_rep='X_pca_harmony')
sc.tl.leiden(N_SC,resolution=1.5)
sc.tl.umap(N_SC)
sc.pl.umap(N_SC, color='sample',save='N_SC_harony_sample.svg')
N_SC.write_h5ad('F:/single cell data/SC/N_SC_HVG.h5ad')

In [ ]:
N_SC=sc.read_h5ad('F:/single cell data/SC/N_SC_HVG.h5ad')
N_SC_leiden = N_SC.obs['leiden']
N_SC_leiden.to_csv('F:/single cell data/SC/N_SC_leiden.csv')
N_SC=sc.read_h5ad('F:/single cell data/SC/N_SC_HVG.h5ad')
N_SC=sc.read_h5ad('F:/single cell data/SC/N_SC_HVG.h5ad')
presubclass = pd.read_csv('F:/single cell data/SC/N_SC_leiden_presubclass.csv',index_col='cell_id')
preclass = pd.read_csv('F:/single cell data/SC/N_SC_leiden_preclass.csv',index_col='cell_id')
N_SC.obs['presubclass'] = presubclass
N_SC.obs['preclass'] = preclass
N_SC.write_h5ad('F:/single cell data/SC/N_SC_pre.h5ad')
sc.pl.umap(N_SC, color='presubclass', groups='PAG', save='N_SC_pre_PAG.svg')

In [ ]:
presubclass = pd.read_csv('F:/single cell data/SC/N_SC_leiden_presubclass.csv',index_col='cell_id')
N_SC.obs['presubclass'] = presubclass
N_SC = N_SC[N_SC.obs['leiden']!='1']

N_SC.write_h5ad('F:/single cell data/SC/N_SC_NOPAG.h5ad')
print(N_SC)
sc.pl.umap(N_SC, color='leiden', legend_loc='on data', legend_fontsize='5', save='N_SC_NOPAG.svg')

In [ ]:
N_SC=sc.read_h5ad('F:/single cell data/SC/N_SC_NOPAG.h5ad')
cluster_dict= {0:'Oligo A',2:'Astro', 3:'SCsg Gabrr2 Gaba A',4:'SCs Dmbx1 Gaba A',5:'SCsg Pde5a Glut A',6:'SC Lef1 Otx2 Gaba',
               7:'SC-PAG Lef1 Emx2 Gaba A',8:'SCig Foxb1 Glut A',9:'SCs Pax7 Nfia Gaba A',10:'SC Bnc2 Glut',11:'SC-PAG Lef1 Emx2 Gaba B',12:'OPC',
               13:'Microglia',14:'SCiw Pitx2 Glut',15:'SCs Dmbx1 Gaba B',16:'SCs Pax7 Nfia Gaba B',17:'Oligo B',18:'SCsg Pde5a Glut B',19:'SCsg Pde5a Glut C',
               20:'SCsg Gabrr2 Gaba B',21:'PAG-SC Neurod2 Meis2 Glut',22:'SCs Dmbx1 Gaba C',23:'SCig Foxb1 Glut B',24:'SCs Lef1 Gli3 Gaba A',
               25:'SCm-PAG Cdh23 Gaba',26:'SC Tnnt1 Gli3 Gaba A',27:'SCop Sln Glut',28:'SCig Tfap2b Chrnb3 Glut A',29:'SC Otx2 Gcnt4 Gaba',
               30:'SCs Lef1 Gli3 Gaba B',31:'SC Tnnt1 Gli3 Gaba B',32:'SCs Pax7 Nfia Gaba C',33:'SCs Dmbx1 Gaba D',34:'VLMC-ABC',35:'Oligo C',
               36:'SCsg Gabrr2 Gaba C',37:'BAM',38:'SCig Tfap2b Chrnb3 Glut B',39:'SCs Pax7 Nfia Gaba D'}
cluster = []
for i in cluster_dict:cluster.append((cluster_dict[i]))
N_SC.obs['cluster']= N_SC.obs['leiden']
N_SC.rename_categories(key='cluster', categories=cluster)
N_SC.write_h5ad('F:/single cell data/SC/N_SC_pre_NOPAG_cluster.h5ad')

In [ ]:
N_SC=sc.read_h5ad('F:/single cell data/SC/N_SC_pre_NOPAG_cluster.h5ad')
subclass =[]
for i in range(N_SC.shape[0]):
    if (int(N_SC.obs['leiden'][i]) in [0,17,35]):
        subclass.append('Oligo')
    elif (int(N_SC.obs['leiden'][i]) in [3,20,36]):
        subclass.append('SCsg Gabrr2 Gaba')
    elif (int(N_SC.obs['leiden'][i]) in [5,18,19]):
        subclass.append('SCsg Pde5a Glut')
    elif (int(N_SC.obs['leiden'][i]) in [7,11]):
        subclass.append('SC-PAG Lef1 Emx2 Gaba')
    elif (int(N_SC.obs['leiden'][i]) in [8,23]):
        subclass.append('SCig Foxb1 Glut')
    elif (int(N_SC.obs['leiden'][i]) in [28,38]):
        subclass.append('SCig Tfap2b Chrnb3 Glut')
    elif (int(N_SC.obs['leiden'][i]) in  [4,15,22,33]):
        subclass.append('SCs Dmbx1 Gaba')
    elif (int(N_SC.obs['leiden'][i]) in  [9,16,32,39]):
        subclass.append('SCs Pax7 Nfia Gaba')
    elif (int(N_SC.obs['leiden'][i]) in  [24,30]):
        subclass.append('SCs Lef1 Gli3 Gaba')
    elif (int(N_SC.obs['leiden'][i]) in  [26,31]):
        subclass.append('SC Tnnt1 Gli3 Gaba')
    elif (int(N_SC.obs['leiden'][i]) ==  10):
        subclass.append('SC Bnc2 Glut')
    elif (int(N_SC.obs['leiden'][i]) ==  12):
        subclass.append('OPC')
    elif (int(N_SC.obs['leiden'][i]) ==  13):
        subclass.append('Microglia')
    elif (int(N_SC.obs['leiden'][i]) ==  27):
        subclass.append('SCop Sln Glut')
    elif (int(N_SC.obs['leiden'][i]) ==  34):
        subclass.append('VLMC-ABC')
    elif (int(N_SC.obs['leiden'][i]) ==  37):
        subclass.append('BAM')
    elif (int(N_SC.obs['leiden'][i]) ==  14):
        subclass.append('SCiw Pitx2 Glut')
    elif (int(N_SC.obs['leiden'][i]) ==  25):
        subclass.append('SCm-PAG Cdh23 Gaba')
    elif (int(N_SC.obs['leiden'][i]) ==   21):
        subclass.append('PAG-SC Neurod2 Meis2 Glut')
    elif (int(N_SC.obs['leiden'][i]) ==  6):
        subclass.append('SC Lef1 Otx2 Gaba')
    elif (int(N_SC.obs['leiden'][i]) ==  29):
        subclass.append('SC Otx2 Gcnt4 Gaba')
    elif (int(N_SC.obs['leiden'][i]) == 2):
        subclass.append('Astro')

N_SC.obs['subclass'] = subclass
N_SC=N_SC[N_SC.obs['subclass']!='BAM']
N_SC.write_h5ad('F:/single cell data/SC/N_SC_pre_NOPAG_subclass.h5ad')

In [ ]:
# D1_SC
D1_SC = sc.read_10x_mtx('H:/data1/D1-SC/outs/filtered_feature_bc_matrix',var_names='gene_symbols', cache=False)
print(D1_SC)
D1_SC.write_h5ad('F:/single cell data/SC/D1-SC.h5ad')

In [ ]:
sc.pp.filter_cells(D1_SC, min_genes=200)#cell must have 200 non-zero-count features to stay
sc.pp.filter_genes(D1_SC, min_cells= 8)#gene must be in 5 cells to stay
mito_genes = D1_SC.var_names.str.startswith('mt-')# for each cell compute fraction of counts in mito genes vs. all genes
D1_SC.obs['percent_mito'] = np.sum(D1_SC[:, mito_genes].X, axis =1)/np.sum(D1_SC.X, axis=1) # add each cell's fraction mito accounts as an obs annotation
D1_SC.obs['n_counts'] =D1_SC.X.sum(axis=1) # add the total counts per cell as observations-annotation
D1_SC = D1_SC[D1_SC.obs.percent_mito < 0.01,:]
D1_SC = D1_SC[D1_SC.obs['n_genes']< 4000,:]
D1_SC = D1_SC[D1_SC.obs['n_counts'] < 10000, :]
print(D1_SC)

In [ ]:
#Visualize count matrix before filtering
sc.pl.violin(D1_SC, ['n_genes', 'n_counts','percent_mito'], jitter =0.4, multi_panel= True)
sc.pl.scatter(D1_SC, x= 'n_counts', y= 'percent_mito')
sc.pl.scatter(D1_SC, x= 'n_counts', y= 'n_genes')
counts_matrix = D1_SC.X
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate= 0.06)
doublet_scores, precdicted_doublets = scrub.scrub_doublets(min_cells= 8, min_gene_variability_pctl= 85, n_prin_comps=40)
D1_SC.obs['Doublet'] = scrub.predicted_doublets_
D1_SC.obs['Doublet Score'] = scrub.doublet_scores_obs_
D1_SC = D1_SC[D1_SC.obs['Doublet']== False,:]
D1_SC.write_h5ad('F:/single cell data/SC/D1-SC_filter.h5ad')
print(D1_SC)

In [ ]:
#D2-SC
D2_SC = sc.read_10x_mtx('H:/data1/D2-SC/outs/filtered_feature_bc_matrix',var_names='gene_symbols', cache=False)
print(D2_SC)
D2_SC.write_h5ad('F:/single cell data/SC/D2-SC.h5ad')

In [ ]:
sc.pp.filter_cells(D2_SC, min_genes=200)#cell must have 200 non-zero-count features to stay
sc.pp.filter_genes(D2_SC, min_cells= 8)#gene must be in 5 cells to stay
mito_genes = D2_SC.var_names.str.startswith('mt-')# for each cell compute fraction of counts in mito genes vs. all genes
D2_SC.obs['percent_mito'] = np.sum(D2_SC[:, mito_genes].X, axis =1)/np.sum(D2_SC.X, axis=1) # add each cell's fraction mito accounts as an obs annotation
D2_SC.obs['n_counts'] =D2_SC.X.sum(axis=1) # add the total counts per cell as observations-annotation
D2_SC = D2_SC[D2_SC.obs.percent_mito < 0.01,:]
D2_SC = D2_SC[D2_SC.obs['n_genes']< 4000,:]
D2_SC = D2_SC[D2_SC.obs['n_counts'] < 10000, :]
print(D2_SC)

In [ ]:
#Visualize count matrix before filtering
sc.pl.violin(D2_SC, ['n_genes', 'n_counts','percent_mito'], jitter =0.4, multi_panel= True)
sc.pl.scatter(D2_SC, x= 'n_counts', y= 'percent_mito')
sc.pl.scatter(D2_SC, x= 'n_counts', y= 'n_genes')
counts_matrix = D2_SC.X
scrub = scr.Scrublet(counts_matrix, expected_doublet_rate= 0.06)
doublet_scores, precdicted_doublets = scrub.scrub_doublets(min_cells= 8, min_gene_variability_pctl= 85, n_prin_comps=40)
D2_SC.obs['Doublet'] = scrub.predicted_doublets_
D2_SC.obs['Doublet Score'] = scrub.doublet_scores_obs_
D2_SC = D2_SC[D2_SC.obs['Doublet']== False,:]
D2_SC.write_h5ad('F:/single cell data/SC/D2-SC_filter.h5ad')
print(D2_SC)

In [ ]:
#D1_SC&D2_SC
D1_SC=sc.read_h5ad('F:/single cell data/SC/D1-SC_filter.h5ad')
D2_SC=sc.read_h5ad('F:/single cell data/SC/D2-SC_filter.h5ad')
D_SC =sc.AnnData.concatenate(D1_SC, D2_SC,batch_categories=['D1_SC','D2_SC'],batch_key='sample')
sc.pp.normalize_per_cell(D_SC,counts_per_cell_after= 1e4)
sc.pp.log1p(D_SC)
sc.pp.highly_variable_genes(D_SC, min_mean=0.0125, max_mean=3, min_disp=0.5)
D_SC=sc.read_h5ad('F:/single cell data/SC/D_SC_HVG.h5ad')
presubclass = pd.read_csv('F:/single cell data/SC/D_SC_leiden_presubclass.csv',index_col='cell_id')
preclass = pd.read_csv('F:/single cell data/SC/D_SC_leiden_preclass.csv',index_col='cell_id')
D_SC.obs['presubclass'] = presubclass
D_SC.obs['preclass'] = preclass
sc.tl.pca(D_SC,svd_solver='arpack')
sc.pl.pca_variance_ratio(D_SC, log=True)
sc.external.pp.harmony_integrate(D_SC,key='sample')
sc.pp.neighbors(D_SC,n_neighbors=25, n_pcs=40,use_rep='X_pca_harmony')
sc.tl.leiden(D_SC,resolution=1.5)
sc.tl.umap(D_SC)
D_SC.write_h5ad('F:/single cell data/SC/D_SC_pre.h5ad')

In [ ]:
D_SC=sc.read_h5ad('F:/single cell data/SC/D_SC_pre.h5ad')
D_SC_leiden = D_SC.obs['leiden']
D_SC_leiden.to_csv('F:/single cell data/SC/D_SC_leiden.csv')
D_SC = D_SC[D_SC.obs['leiden']!='1']
D_SC = D_SC[D_SC.obs['leiden']!='17']
D_SC.write_h5ad('F:/single cell data/SC/D_SC_NOPAG.h5ad')
print(D_SC.obs['leiden'])

In [ ]:
cluster_dict= {0:'Oligo 1', 2:'Astro',3:'SC-PAG Lef1 Emx2 Gaba',4:'SCs Dmbx1 Gaba 1',5:'SCsg Gabrr2 Gaba 1',6:'SCsg Pde5a Glut 1',
               7:'SCs Pax7 Nfia Gaba 1',8:'SC Lef1 Otx2 Gaba',9:'SCig Foxb1 Glut 1',10:'SC Bnc2 Glut',11:'SCiw Pitx2 Glut',12:'OPC',
               13:'SCsg Gabrr2 Gaba 2',14:'SCig Foxb1 Glut 2',15:'SCig Foxb1 Glut 3',16:'SCsg Pde5a Glut 2',18:'PAG-SC Neurod2 Meis2 Glut',
               19:'SCsg Pde5a Glut 3',20:'SCs Lef1 Gli3 Gaba 1',21:'Oligo 2',22:'SCs Dmbx1 Gaba 2',23:'SCm-PAG Cdh23 Gaba',
               24:'SCop Sln Glut',25:'SCig Tfap2b Chrnb3 Glut',26:'SCs Dmbx1 Gaba 3',27:'Microglia',28:'SC Otx2 Gcnt4 Gaba',29:'SCs Pax7 Nfia Gaba 2',
               30:'SC Tnnt1 Gli3 Gaba 1',31:'SCs Dmbx1 Gaba 4',32:'SCs Lef1 Gli3 Gaba 2',
               33:'SC Tnnt1 Gli3 Gaba 2',34:'SCs Pax7 Nfia Gaba 3',35:'Oligo 3',36:'VLMC-ABC',37:'Oligo 4',38:'Oligo 5'}
cluster = []
for i in cluster_dict:cluster.append((cluster_dict[i]))
D_SC.obs['cluster']= D_SC.obs['leiden']
D_SC.rename_categories(key='cluster', categories=cluster)
D_SC.write_h5ad('F:/single cell data/SC/D_SC_NOPAG_pre.h5ad')

In [ ]:
SC=sc.read_h5ad('F:/single cell data/SC/D_SC_NOPAG_pre.h5ad')
subclass =[]
for i in range(SC.shape[0]):
    if (int(SC.obs['leiden'][i]) in [0,21,35,37,38]):
        subclass.append('Oligo')
    elif (int(SC.obs['leiden'][i]) in [5,13]):
        subclass.append('SCsg Gabrr2 Gaba')
    elif (int(SC.obs['leiden'][i]) in [6,16,19]):
        subclass.append('SCsg Pde5a Glut')
    elif (int(SC.obs['leiden'][i]) in [9,14,15]):
        subclass.append('SCig Foxb1 Glut')
    elif (int(SC.obs['leiden'][i]) in  [7,29,34]):
        subclass.append('SCs Pax7 Nfia Gaba')
    elif (int(SC.obs['leiden'][i]) in  [4,22,26,31]):
        subclass.append('SCs Dmbx1 Gaba')
    elif (int(SC.obs['leiden'][i]) in  [20,32]):
        subclass.append('SCs Lef1 Gli3 Gaba')
    elif (int(SC.obs['leiden'][i]) in  [30,33]):
        subclass.append('SC Tnnt1 Gli3 Gaba')
    elif (int(SC.obs['leiden'][i]) ==  8):
        subclass.append('SC Lef1 Otx2 Gaba')
    elif (int(SC.obs['leiden'][i]) ==  28):
        subclass.append('SC Otx2 Gcnt4 Gaba')
    elif (int(SC.obs['leiden'][i]) ==  12):
        subclass.append('OPC')
    elif (int(SC.obs['leiden'][i]) ==  27):
        subclass.append('Microglia')
    elif (int(SC.obs['leiden'][i]) ==  24):
        subclass.append('SCop Sln Glut')
    elif (int(SC.obs['leiden'][i]) ==  36):
        subclass.append('VLMC-ABC')
    elif (int(SC.obs['leiden'][i]) ==  2):
        subclass.append('Astro')
    elif (int(SC.obs['leiden'][i]) == 11 ):
        subclass.append('SCiw Pitx2 Glut')
    elif (int(SC.obs['leiden'][i]) == 3):
        subclass.append('SC-PAG Lef1 Emx2 Gaba')
    elif (int(SC.obs['leiden'][i]) == 25):
        subclass.append('SCig Tfap2b Chrnb3 Glut')
    elif (int(SC.obs['leiden'][i]) ==  10):
        subclass.append('SC Bnc2 Glut')
    elif (int(SC.obs['leiden'][i]) == 23):
        subclass.append('SCm-PAG Cdh23 Gaba')
    elif (int(SC.obs['leiden'][i]) == 18):
        subclass.append('PAG-SC Neurod2 Meis2 Glut')
SC.obs['subclass'] = subclass
SC.write_h5ad('F:/single cell data/SC/D_SC_pre_NOPAG_subclass.h5ad')